# Set up

In [1]:
!pip install -U ffmpeg youtube-dl cuda torch_optimizer
#! pip install accelerate cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
#! pip install git+https://github.com/huggingface/accelerate

     |████████████████████████████████| 1.9 MB 25.8 MB/s 
     |████████████████████████████████| 72 kB 1.1 MB/s 
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=4f75694992476e902a135d66e46c5bb777ad0c48a34aca8b00bf54ee3ab46450
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
  Created wheel for cuda: filename=cuda-0.0.1-py3-none-any.whl size=1170 sha256=0c0422fdd3efefaa5da887fbdbb8eb100d41d3c475e3fac4f376b736e387ef1f
  Stored in directory: /root/.cache/pip/wheels/58/66/7d/152049687c7376a7681a770e99f46ab09d45098826ecc4941d
Successfully built ffmpeg cuda


In [2]:
!git clone https://github.com/commaai/speedchallenge.git
!rm speedchallenge/data/train.mp4
!youtube-dl -f mp4 -o speedchallenge/data/train.mp4 https://youtu.be/LMJHVc3fpo4
#!rm speedchallenge/data/test.mp4
#!youtube-dl -f mp4 -o speedchallenge/data/test.mp4 https://youtu.be/FcAbi_mutTA

Cloning into 'speedchallenge'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 14 (delta 0), reused 0 (delta 0), pack-reused 11
Unpacking objects: 100% (14/14), done.
[youtube] LMJHVc3fpo4: Downloading webpage
[youtube] LMJHVc3fpo4: Downloading MPD manifest
[download] Destination: speedchallenge/data/train.mp4
[download] 100% of 49.39MiB in 26:54
[youtube] FcAbi_mutTA: Downloading webpage
[youtube] FcAbi_mutTA: Downloading MPD manifest
[download] Destination: speedchallenge/data/test.mp4
[download] 100% of 28.50MiB in 00:01


In [3]:
!mkdir clean-images
!mkdir assets
!mkdir clean-images/train
!mkdir clean-images/test
!mkdir clean-images/processed

# Imports

In [4]:
import numpy as np
import tensorflow as tf
import cv2
import csv
import os
import copy
import shutil
import time
import subprocess
import string
import random
from google.colab import drive
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import pandas as pd
from PIL import Image 
import ipywidgets as widgets
from IPython.html.widgets import *
import pickle
import h5py
import torch
from torch import nn, flatten
from torch.autograd import Variable
from torch.nn import Module, Linear, Conv2d, MaxPool2d, Dropout, ELU, MSELoss, Sequential
from torch.optim import SGD, Adam
from torch.utils.data import DataLoader
import torchvision as tv
from torch_optimizer import DiffGrad, NovoGrad, SWATS, Yogi, AdaBound

/usr/local/lib/python3.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [5]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Preprocessing

In [6]:
def change_brightness(image, bright_factor):
    """
    Augments the brightness of the image by multiplying the saturation by a uniform random variable
    Input: image (RGB)
    returns: image with brightness augmentation
    """
    
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    # perform brightness augmentation only on the second channel
    hsv_image[:,:,2] = hsv_image[:,:,2] * bright_factor
    
    # change back to RGB
    image_rgb = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2RGB)
    return image_rgb

In [7]:
'''
This needs to be modified to account for input images of all sizes
'''
def resize_image(image):
    """
    preprocesses the image
    
    input: image (480 (y), 640 (x), 3) RGB
    output: image (shape is (220, 66, 3) as RGB)
    
    This stuff is performed on my validation data and my training data
    Process: 
             1) Cropping out black spots
             3) resize to (220, 66, 3) if not done so already from perspective transform
    """
    # Crop out sky (top) (100px) and black right part (-90px)
    #image_cropped = image[100:440, :-90] # -> (380, 550, 3) #v2 for data
    image_cropped = image[25:375, :] #v1 for data
    
    image = cv2.resize(image_cropped, (220, 66), interpolation = cv2.INTER_AREA)
    
    return image

In [8]:
def preprocess_image_from_path(image_path, speed, bright_factor):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = change_brightness(img, bright_factor)    
    img = resize_image(img)
    return img, speed

# Lucas-Kanade method with pyramids Optical Flow

In [9]:
def opticalFlowSparse(image_current, image_next):
  gray1 = cv2.cvtColor(image_current, cv2.COLOR_BGR2GRAY)
  gray2 = cv2.cvtColor(image_next, cv2.COLOR_BGR2GRAY)
  frame = image_current

  #ShiTomasi corner detection
  p0 = cv2.goodFeaturesToTrack(gray1, mask=None, maxCorners=100, qualityLevel=0.3, \
                               minDistance=7, blockSize=7)
  #assert p0 is not None, 'Sparse optical flow failed?'
  if p0 is None:
    return np.zeros((frame.shape[0], frame.shape[1], 3), np.uint8)
  
  p1, st, err = cv2.calcOpticalFlowPyrLK(gray1, gray2, p0, None, winSize=(15, 15), \
                                         maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS \
                                                               | cv2.TERM_CRITERIA_COUNT, \
                                                               10, 0.03))
  mask = np.zeros_like(gray1)
  color = np.random.randint(0, 255, (100, 3))
  good_new = p1[st == 1]
  good_old = p0[st == 1]
  for i, (new, old) in enumerate(zip(good_new, good_old)):
    a, b = new.ravel()
    c, d = old.ravel()
    mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), 2)
    frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1)

  result = cv2.bitwise_and(frame, frame, mask=mask)
  return result

# Dataset Construction

In [10]:
def video_to_frames(video_path, img_folder):
  class getFrames():
    def __init__(self):
      self.vid = cv2.VideoCapture(video_path)
    def __iter__(self):
      return self
    def __next__(self):
      con, frame = self.vid.read()
      if con:
        return frame
      else:
        raise StopIteration
    def __len__(self):
      return int(self.vid.get(cv2.CAP_PROP_FRAME_COUNT))

  frame_paths = list()
  for idx, frame in enumerate(tqdm(getFrames(), 'Converting video to still frames')):
    img_path = os.path.join(img_folder, str(idx)+'.jpg')
    cv2.imwrite(img_path, frame) 
    frame_paths.append(img_path)
  
  return frame_paths

In [11]:
class Dataset_Custom(torch.utils.data.Dataset):
  def __init__(self, image_paths, speeds, preprocessing_step, optical_flow_step, vary_brightness=False):
    super(Dataset_Custom).__init__()
    assert len(image_paths) == len(speeds), 'lengths {}, {} do not match'.format(len(image_paths), len(speeds))
    self.image_paths = image_paths
    self.speeds = speeds
    self.preprocessing_step = preprocessing_step
    self.optical_flow_step = optical_flow_step
    self.vary_brightness = vary_brightness

  def __len__(self):
    return len(self.speeds) - 1

  def __getitem__(self, idx):
    if idx < 0 or idx >= len(self):
      raise IndexError('index {} is out of bounds (len={})'.format(idx, len(self)))

    image_now = self.image_paths[idx]
    image_next = self.image_paths[idx + 1]

    speed_now = self.speeds[idx]
    speed_next = self.speeds[idx + 1]


    bright_factor = 1.0
    if self.vary_brightness:
      bright_factor = 0.2 + np.random.uniform()

    x1, y1 = self.preprocessing_step(image_now, speed_now, bright_factor)
    x2, y2 = self.preprocessing_step(image_next, speed_next, bright_factor)

    rgb_diff = self.optical_flow_step(x1, x2)
    y = np.mean([y1, y2])

    return (tv.transforms.ToTensor()(rgb_diff), float(y))

In [12]:
def preprocess_images(image_paths, target_dir, speeds, preprocessing_step, optical_flow_step, vary_brightness=False):
  for idx in tqdm(range(len(image_paths) - 1), 'Preprocessing Images'):
    image_now = image_paths[idx]
    image_next = image_paths[idx + 1]
    
    speed_now = speeds[idx]
    speed_next = speeds[idx + 1]
    
    bright_factor = 1.0
    if vary_brightness:
      bright_factor = 0.2 + np.random.uniform()

    x1, y1 = preprocessing_step(image_now, speed_now, bright_factor)
    x2, y2 = preprocessing_step(image_next, speed_next, bright_factor)

    rgb_diff = optical_flow_step(x1, x2)
    new_path = '{}/image_{}.png'.format(target_dir, idx)
    cv2.imwrite(new_path, rgb_diff)

    y = np.mean([y1, y2])

    yield y, new_path

In [13]:
class Dataset_Custom_Fast(torch.utils.data.Dataset):
  def __init__(self, image_paths, speeds):
    super(Dataset_Custom).__init__()
    assert len(image_paths) == len(speeds), 'lengths {}, {} do not match'.format(len(image_paths), len(speeds))
    self.image_paths = image_paths
    self.speeds = speeds

  def __len__(self):
    return len(self.speeds)

  def __getitem__(self, idx):
    if idx < 0 or idx >= len(self):
      raise IndexError('index {} is out of bounds (len={})'.format(idx, len(self)))

    return (tv.transforms.ToTensor()(cv2.imread(self.image_paths[idx])), float(self.speeds[idx]))

In [14]:
TRAIN_RATIO = 0.7

comma_frames = video_to_frames('speedchallenge/data/train.mp4', 'clean-images/train')
comma_speeds = list(pd.read_csv('speedchallenge/data/train.txt', header=None, squeeze=True))
comma_speeds, comma_frames = zip(*preprocess_images(comma_frames, 'clean-images/processed', \
                                                    comma_speeds, preprocess_image_from_path, \
                                                    opticalFlowSparse, True))

comma_frames_train, comma_frames_valid, comma_speeds_train, comma_speeds_valid = train_test_split(comma_frames, comma_speeds, train_size=TRAIN_RATIO)

Converting video to still frames:   0%|          | 0/20400 [00:00<?, ?it/s]

Preprocessing Images:   0%|          | 0/20399 [00:00<?, ?it/s]

# NVIDIA CNN

In [15]:
class nvidia_cnn(Module):
  def __init__(self):
    super(nvidia_cnn, self).__init__()

    self.conv1 = Conv2d(3, 24,
        kernel_size=(5, 5),
        stride=(2, 2),
        padding='valid'
        )
    self.elu1 = ELU()
    self.conv2 = Conv2d(24, 36,
        kernel_size=(5, 5),
        stride=(2,2),
        padding='valid'
    )
    self.elu2 = ELU()
    self.conv3 = Conv2d(36, 48,
        kernel_size=(5, 5),
        stride=(2, 2),
        padding='valid',
    )
    self.elu3 = ELU()
    self.dropout1 = Dropout(0.5)
    self.conv4 = Conv2d(48, 64,
        kernel_size=(3, 3),
        stride=(1, 1),
        padding='valid'
    )
    self.elu4 = ELU()
    self.conv5 = Conv2d(64, 64,
        kernel_size=(3, 3),
        stride=(1, 1),
        padding='valid'
    )
    self.elu5 = ELU()
    self.fc0 = Linear(1280, 100)
    self.elu6 = ELU()
    self.fc1 = Linear(100, 50)
    self.elu7 = ELU()
    self.fc2 = Linear(50, 10)
    self.elu8 = ELU()
    self.fc3 = Linear(10, 1)
    self.elu9 = ELU()

  def forward(self, x):
    x = self.conv1(x)
    x = self.elu1(x)
    x = self.conv2(x)
    x = self.elu2(x)
    x = self.conv3(x)
    x = self.elu3(x)
    x = self.dropout1(x)
    x = self.conv4(x)
    x = self.elu4(x)
    x = self.conv5(x)
    x = flatten(x, 1)
    x = self.elu5(x)
    x = self.fc0(x)
    x = self.elu6(x)
    x = self.fc1(x)
    x = self.elu7(x)
    x = self.fc2(x)
    x = self.elu7(x)
    x = self.fc3(x)
    x = self.elu9(x)
    x = flatten(x)
    return x

# Training

In [ ]:
EPOCHS = 30
NUM_BATCHES = 64 #Not implemented
BATCH_SIZE = 8

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device is {}'.format(device))


history = {}

#train_loader = Dataset_Custom( \
#    comma_frames_train, comma_speeds_train, preprocess_image_from_path, \
#    opticalFlowDense, True
#    )
train_loader = Dataset_Custom_Fast(comma_frames_train, comma_speeds_train)
train_loader = DataLoader(train_loader, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

#valid_loader = Dataset_Custom( \
#    comma_frames_valid, comma_speeds_valid, preprocess_image_from_path, \
#    opticalFlowDense, True
#    )
valid_loader = Dataset_Custom_Fast(comma_frames_valid, comma_speeds_valid)
valid_loader = DataLoader(valid_loader, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)


model = nvidia_cnn()
criterion = MSELoss()

use_cuda = torch.cuda.is_available()
if use_cuda:
  model = model.cuda()

optim = Adam(
    model.parameters(),
    lr=1e-4,
    betas=(0.9,0.999),
    eps=1e-08,
    weight_decay=0.0
    )

#optim = DiffGrad( #Stalls out at ~10MSE
#    model.parameters(),
#    lr= 1e-4,
#    betas=(0.9, 0.999),
#    eps=1e-8,
#    weight_decay=0,
#)

#optim = NovoGrad(
#    model.parameters(),
#    lr= 1e-4,
#    betas=(0.9, 0.999),
#    eps=1e-8,
#    weight_decay=0,
#    grad_averaging=False,
#    amsgrad=False,
#)

#optim = SWATS( #Bad
#    model.parameters(),
#    lr=1e-4,
#    betas=(0.9, 0.999),
#    eps=1e-8,
#    weight_decay= 0.0,
#    amsgrad=False,
#    nesterov=False,
#)

#optim = Yogi( #Stops generalizing at about 21MSE
#    model.parameters(),
#    lr= 1e-4,
#    betas=(0.9, 0.999),
#    eps=1e-8,
#    initial_accumulator=1e-6,
#    weight_decay=0,
#)

#optim = AdaBound( #Stalls out at around 200MSE
#    model.parameters(),
#    lr= 1e-4,
#    betas= (0.9, 0.999),
#    final_lr = 0.1,
#    gamma=1e-3,
#    eps= 1e-8,
#    weight_decay=0,
#    amsbound=False,
#)


start_n_iter = 0
start_epoch = 0

model = torch.nn.DataParallel(model)

for epoch in range(start_epoch + 1, EPOCHS + 1):  
  model.train()

  pbar = tqdm(enumerate(train_loader), total=len(train_loader), dynamic_ncols=True)
  history[epoch] = {}
  start_time = time.time()

  total_loss = 0.0
  denom = 0
  optim.zero_grad()
  for i, data in pbar:
    img, label = data
    if use_cuda:
      img = img.cuda()
      label = label.cuda()
    
    prepare_time = start_time - time.time()


    out = model(img)
    loss = criterion(out, label.float())

    loss.backward()
    cur_loss = loss.item()
    total_loss += cur_loss
    denom += 1

    process_time = start_time - time.time() - prepare_time
    pbar.set_description('eff: {:.2f}%, loss:{:.2f}, {:.2f}, epoch: {}/{}:'.format(
        100.0*process_time/(process_time + prepare_time), cur_loss, total_loss/denom, epoch, EPOCHS
    ))
    start_time = time.time()

    optim.step()
    optim.zero_grad()
  total_loss /= denom
  history[epoch]['training loss'] = total_loss
  history[epoch]['weights'] = copy.deepcopy(model.state_dict())
  print('Epoch: {}, Total loss (training): {}'.format(epoch, total_loss))

  model.eval()
  pbar = tqdm(enumerate(valid_loader), total=len(valid_loader), dynamic_ncols=True)
  total_loss = 0.0
  denom = 0
  for i, data in pbar:
    img, label = data
    if use_cuda:
      img.cuda()
      label.cuda()

    with torch.no_grad(): 
      out = model(img)
    loss = criterion(out.to(device=device), label.float().to(device=device))
    total_loss += loss.item()
    denom += 1
  history[epoch]['validation loss'] = total_loss/denom
  print('Epoch: {}, Total loss (validation): {}\n\n'.format(epoch, total_loss/denom))

device is cpu


  0%|          | 0/1784 [00:00<?, ?it/s]

Epoch: 1, Total loss (training): 70.78084443716725


  0%|          | 0/765 [00:00<?, ?it/s]

Epoch: 1, Total loss (validation): 66.38100847107133




...

  0%|          | 0/1784 [00:00<?, ?it/s]

Epoch: 17, Total loss (training): 6.468847286950112


  0%|          | 0/765 [00:00<?, ?it/s]

Epoch: 17, Total loss (validation): 8.886167222378301




  0%|          | 0/1784 [00:00<?, ?it/s]

In [ ]:
for epoch in history.keys():
  for key in history[epoch]['weights'].keys():
    history[epoch]['weights'][key] = history[epoch]['weights'][key].cpu().numpy().tolist()

In [ ]:
with open('history.json', 'w') as f:
  json.dump(history, f)

In [ ]:
!zip -9 history.json.zip history.json

In [ ]:
!mv history.json.zip gdrive/MyDrive/"CSP572: CCC IS"/"Individual Topics"/'Speed Detection'/history_m3_lr1e-4.json.zip

In [ ]:
for epoch in history.keys():
  for key in history[epoch]['weights'].keys():
    history[epoch]['weights'][key] = torch.tensor(history[epoch]['weights'][key])

#Visualization

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(15, 11))
x = list(history.keys())
c = (['training loss',] * len(x)) + (['validation loss',] * len(x))
x += x
y = [history[epoch][typ] for epoch, typ in zip(x, c)]
sns.lineplot(x=x, y=y, hue=c, ax=axs)
sns.scatterplot(x=x, y=y, hue=c, ax=axs, legend=False)
axs.set_title('Loss During Training (MSE in MPH)')
axs.set_xlabel('Epoch')
axs.set_ylabel('MSE')
plt.savefig('history.jpg')
plt.show()

In [ ]:
data_loader = Dataset_Custom_Fast(comma_frames, comma_speeds)
data_loader = DataLoader(data_loader, batch_size=1, shuffle=False, drop_last=False)

model.eval()
pbar = tqdm(enumerate(data_loader), 'Making Predictions', total=len(data_loader), dynamic_ncols=True)
results = {
    'prediction':[],
    'actual':comma_speeds
}
for i, data in pbar:
  img, label = data
  if use_cuda:
    img.cuda()
    label.cuda()

  with torch.no_grad(): 
    #out = model(img, device)
    results['prediction'].insert(i, model(img))

for i in tqdm(range(len(results['prediction'])), 'Converting'):
  results['prediction'][i] = results['prediction'][i].cpu().tolist()[0]

#format for display
def tmp():
  for i in range(len(results['prediction'])):
    yield i, results['prediction'][i], 'Prediction'
  for i in range(len(results['actual'])):
    yield i, results['actual'][i], 'Actual'
x, y, c = zip(*tmp())

fig, axs = plt.subplots(1, 1, figsize=(15, 11))
sns.scatterplot(x=x, y=y, hue=c, ax=axs, legend=True)
#sns.lineplot(x=x, y=y, hue=c, ax=axs, legend=True)
axs.set_xlabel('Time')
axs.set_ylabel('Speed')
plt.savefig('comparison.jpg')
plt.show()

In [ ]:
def tmp():
  for i in range(len(results['prediction'])):
    yield i, results['prediction'][i] - results['actual'][i]
t, err = zip(*tmp())

fig, axs = plt.subplots(1, 1, figsize=(15, 11))
sns.scatterplot(x=t, y=err, ax=axs)
axs.set_xlabel('Time')
axs.set_ylabel('Error')
axs.set_title('Error vs. Time')
plt.show()

In [ ]:
def tmp():
  for i in range(len(results['prediction'])):
    yield i, np.abs(results['prediction'][i] - results['actual'][i])
t, err = zip(*tmp())

fig, axs = plt.subplots(1, 1, figsize=(15, 11))
sns.scatterplot(x=t, y=err, ax=axs)
axs.set_xlabel('Time')
axs.set_ylabel('Absolute Error')
axs.set_title('Absolute Error vs. Time')
plt.show()

In [ ]:
def tmp():
  for i in range(len(results['prediction'])):
    yield results['actual'][i], results['prediction'][i] - results['actual'][i]
t, err = zip(*tmp())

fig, axs = plt.subplots(1, 1, figsize=(15, 11))
sns.scatterplot(x=t, y=err, ax=axs)
axs.set_xlabel('Actual Speed')
axs.set_ylabel('Error')
axs.set_title('Error vs. Actual Speed')
plt.show()

In [ ]:
def tmp():
  for i in range(len(results['prediction'])):
    yield results['actual'][i], np.abs(results['prediction'][i] - results['actual'][i])
t, err = zip(*tmp())

fig, axs = plt.subplots(1, 1, figsize=(15, 11))
sns.scatterplot(x=t, y=err, ax=axs)
axs.set_xlabel('Actual Speed')
axs.set_ylabel('Absolute Error')
axs.set_title('Absolute Error vs. Actual Speed')
plt.show()

In [ ]:
def tmp():
  for i in range(len(results['prediction'])):
    yield results['actual'][i], np.abs(results['prediction'][i] - results['actual'][i])
t, err = zip(*tmp())

fig, axs = plt.subplots(1, 1, figsize=(15, 11))
sns.kdeplot(x=t, y=err, ax=axs)
axs.set_xlabel('Actual Speed')
axs.set_ylabel('Absolute Error')
axs.set_title('Absolute Error vs. Actual Speed Distribution')
plt.show()

In [ ]:
def tmp():
  for i in range(len(results['prediction'])):
    yield results['actual'][i], np.abs(results['prediction'][i] - results['actual'][i])
t, err = zip(*tmp())

fig, axs = plt.subplots(1, 1, figsize=(15, 11))
sns.kdeplot(x=err, ax=axs)
#axs.set_xlabel('Actual Speed')
axs.set_xlabel('Absolute Error')
axs.set_title('Absolute Error Distribution')
plt.show()